In [1]:
import os
from glob import glob
from PIL import Image, UnidentifiedImageError ,ImageFile 
import numpy as np
from sentence_transformers import SentenceTransformer


ImageFile.LOAD_TRUNCATED_IMAGES=True

2025-10-13 03:13:19.870051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760325200.100169      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760325200.163409      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import torch
print(torch.cuda.is_available())   # Should print True
print(torch.cuda.current_device()) # Should print 0
x = torch.randn(2,2).cuda()
print(x.device)                     # Should print cuda:0


True
0
cuda:0


In [3]:

import torch
print("CUDA Available:", torch.cuda.is_available())
print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(0))


CUDA Available: True
Current Device: 0
Device Name: Tesla P100-PCIE-16GB


In [4]:

def generate_clip_embeddings_batched(images_path, model, batch_size=16):
    # Gather all supported image files
    image_paths = (
        glob(os.path.join(images_path, '*.jpg')) +
        glob(os.path.join(images_path, '*.jpeg')) +
        glob(os.path.join(images_path, '*.png'))
    )

    print(f"🔍 Found {len(image_paths)} images in {images_path}")
    if not image_paths:
        raise ValueError("No images found in the given path!")

    embeddings, filenames = [], []
    batch_images, batch_paths = [], []

    for idx, img_path in enumerate(image_paths):
        try:
            # Try opening and fully loading the image
            with Image.open(img_path) as image:
                image.load()  # Ensures truncated/corrupted files raise error immediately
                image = image.convert("RGB")
        except (FileNotFoundError, UnidentifiedImageError, OSError, ValueError) as e:
            print(f"⚠️ Skipping image: {img_path} — {e}")
            continue  # Skip bad images safely

        # Add valid image to batch
        batch_images.append(image)
        batch_paths.append(img_path)

        # Process the batch when it's full or last image reached
        if len(batch_images) == batch_size or idx == len(image_paths) - 1:
            try:
                batch_embeddings = model.encode(
                    batch_images,
                    convert_to_tensor=True,
                    device='cuda',
                    show_progress_bar=True
                )
            except Exception as e:
                print(f"⚠️ Error encoding batch: {e}")
                batch_images, batch_paths = [], []  # Skip batch and continue
                continue

            # Store embeddings and filenames
            embeddings.extend(batch_embeddings.cpu().numpy())
            filenames.extend(batch_paths)

            # Clear for next batch
            batch_images, batch_paths = [], []

    if not embeddings:
        raise ValueError("No valid embeddings generated — all images may be corrupted or missing.")

    # Combine all embeddings and filenames
    embeddings_array = np.vstack(embeddings)
    filenames_array = np.array(filenames)

    return embeddings_array, filenames_array

In [5]:
IMAGES_PATH = '/kaggle/input/ml-challenge-images-train/train-images'
# print(image.devic
model = SentenceTransformer('clip-ViT-B-32')
model.to('cuda')

# Generate embeddings and filenames
embeddings, filenames = generate_clip_embeddings_batched(IMAGES_PATH, model, batch_size=256)

# Save both embeddings and filenames together in one .npz file
np.savez('clip_image_embeddings_and_filenames.npz', embeddings=embeddings, filenames=filenames)

print(f"Saved {len(filenames)} embeddings for images from {IMAGES_PATH}.")

modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Saved 72287 embeddings for images from /kaggle/input/ml-challenge-images-train/train-images.
